In [1]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import count, col
from pyspark.sql.functions import struct
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()
import pandas as pd
Benefit=spark.read.csv(r"C:\DB\benefits_clean_2014.csv", header=True,inferSchema=True)
Rate=spark.read.csv(r"C:\DB\rate_clean_2014.csv", header=True,inferSchema=True)
Attribute=spark.read.csv(r"C:\DB\attributes_clean_2014.csv", header=True,inferSchema=True)



In [2]:
Attribute=Attribute.select("BusinessYear", "DiseaseManagementProgramsNumber", \
                                  "IssuerActuarialValue", \
                                  "MetalLevel", "PlanId", "WellnessProgramOffered", "NationalNetwork","StandardComponentId")

In [3]:
Benefit=Benefit.drop('PlanId')
Benefit=Benefit.withColumnRenamed("StandardComponentId", "PlanId2").drop("BusinessYear")

In [4]:
Attribute=Attribute.drop('PlanId')
Attribute=Attribute.withColumnRenamed("StandardComponentId", "PlanId3").drop("BusinessYear")

In [5]:
#df=Rate.join(Benefit, Benefit.StandardComponentId == Rate.PlanId, 'inner').select(Rate.PlanId,Rate.IndividualRate, Rate.Age ,Benefit.BusinessYear,Benefit.BenefitName,Benefit.CoinsInnTier1Value,Benefit.CoinsInnTier2Value,Benefit.CoinsOutofNetValue,Benefit.LimitQty)
df=Rate.join(Benefit, Benefit.PlanId2 == Rate.PlanId, 'inner')
df=df.drop('PlanId2')
#df=df.join(Attribute, Attribute.PlanId == df.PlanId, 'inner')


In [6]:
df=df.join(Attribute, Attribute.PlanId3 == df.PlanId, 'inner')
df=df.drop('PlanId3')

In [7]:

df

DataFrame[BusinessYear: int, StateCode: string, IssuerId: int, SourceName: string, IssuerId2: int, PlanId: string, RatingAreaId: int, Tobacco: string, Age: string, IndividualRate: double, IndividualTobaccoRate: double, Couple: double, PrimarySubscriberAndOneDependent: double, PrimarySubscriberAndTwoDependents: double, PrimarySubscriberAndThreeOrMoreDependents: double, CoupleAndOneDependent: double, CoupleAndTwoDependents: double, CoupleAndThreeOrMoreDependents: double, BenefitName: string, EHBVarReason: string, IsCovered: string, IsEHB: string, IsExclFromInnMOOP: string, IsExclFromOonMOOP: string, IsStateMandate: string, IsSubjToDedTier1: string, IsSubjToDedTier2: string, IssuerId: int, IssuerId2: int, LimitQty: double, LimitUnit: string, MinimumStay: double, QuantLimitOnSvc: string, SourceName: string, StateCode: string, StateCode2: string, CopayInnTier1Value: double, CopayInnTier1Unit: string, CopayInnTier2Value: double, CopayInnTier2Unit: string, CopayOutofNetValue: double, CopayOut

In [8]:
df2=df.select("BusinessYear","PlanId","Age","IndividualRate","IsEHB","IsStateMandate","DiseaseManagementProgramsNumber",\
         "IssuerActuarialValue","MetalLevel","WellnessProgramOffered","NationalNetwork","CopayInnTier1Value","CoinsInnTier1Value")

In [9]:
df3=df2.filter(df2.Age!="99")

In [10]:
df4=df3.filter(df3.IsEHB!="-1")

In [11]:
df5=df4.filter(df4.IsStateMandate!="-1")
df6=df5.filter(df5.IssuerActuarialValue!="-1")
df7=df6.filter(df6.MetalLevel!="-1")

In [12]:
df8=df7.filter(df7.WellnessProgramOffered!="-1")
df9=df8.filter(df8.NationalNetwork!="-1")
df10=df9.filter(df9.CopayInnTier1Value!="-1")
df11=df10.filter(df10.CoinsInnTier1Value!="-1")

In [13]:
data=df11.filter(df11.IndividualRate.isNotNull())
data=data.filter(data.IsEHB.isNotNull())
data=data.filter(data.IsStateMandate.isNotNull())
data=data.filter(data.IssuerActuarialValue.isNotNull())
data=data.filter(data.MetalLevel.isNotNull())
data=data.filter(data.WellnessProgramOffered.isNotNull())
data=data.filter(data.NationalNetwork.isNotNull())
data=data.filter(data.CopayInnTier1Value.isNotNull())
data=data.filter(data.CoinsInnTier1Value.isNotNull())

In [14]:
data=data.distinct()

In [15]:
data=data.na.replace("0-20","19","Age")
data=data.na.replace("65 and over","66","Age")
data=data.na.replace("No","0","IsEHB")
data=data.na.replace("Yes","1","IsEHB")
data=data.na.replace("No","0","IsStateMandate")
data=data.na.replace("Yes","1","IsStateMandate")
data=data.na.replace("Bronze","1","MetalLevel")
data=data.na.replace("Silver","2","MetalLevel")
data=data.na.replace("Gold","3","MetalLevel")
data=data.na.replace("Platinum","4","MetalLevel")
data=data.na.replace("High","3","MetalLevel")
data=data.na.replace("Low","1","MetalLevel")
data=data.na.replace("Catastrophic","0","MetalLevel")
data=data.na.replace("NO","0","WellnessProgramOffered")
data=data.na.replace("YES","1","WellnessProgramOffered")
data=data.na.replace("NO","0","NationalNetwork")
data=data.na.replace("YES","1","NationalNetwork")

data=data.na.replace("65 and over","66","Age")

In [16]:
data = data.withColumn("Age",data["Age"].cast('int'))
data = data.withColumn("IsEHB",data["IsEHB"].cast('int'))
data = data.withColumn("IsStateMandate",data["IsStateMandate"].cast('int'))
data = data.withColumn("DiseaseManagementProgramsNumber",data["DiseaseManagementProgramsNumber"].cast('int'))
data = data.withColumn("MetalLevel",data["MetalLevel"].cast('int'))
data = data.withColumn("WellnessProgramOffered",data["WellnessProgramOffered"].cast('int'))
data = data.withColumn("NationalNetwork",data["NationalNetwork"].cast('int'))

In [17]:
data=data.drop("PlanId")
data=data.drop("BusinessYear")

In [23]:
from pyspark.ml.linalg import DenseVector
#input_data= data.rdd.map(lambda x: (x[1],DenseVector(x[0],x[2:])))
#input_data= data.rdd.map(lambda x: (x[1],x[0],DenseVector(x[2:])))
input_data= data.rdd.map(lambda x: (x[1],x[0],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10]))


In [24]:
data_model = spark.createDataFrame(input_data, ["label", "F1","F2","F3","F4","F5","F6","F7","F8","F9","F10"])


In [25]:
train,test=data_model.randomSplit([.0001,.0001],seed=321)
train.show()

+-----+---+---+---+---+-----+---+---+---+----+---+
|label| F1| F2| F3| F4|   F5| F6| F7| F8|  F9|F10|
+-----+---+---+---+---+-----+---+---+---+----+---+
| 75.4| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|1.0|
| 75.4| 19|  0|  1|  7|78.42|  3|  1|  1| 0.0|0.3|
| 75.4| 19|  1|  0|  7|78.42|  3|  1|  1| 0.0|0.3|
| 75.4| 19|  1|  1|  7|78.42|  3|  1|  1| 0.0|0.0|
|76.71| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|0.0|
|76.71| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|0.3|
|76.71| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|1.0|
|76.71| 19|  0|  1|  7|78.42|  3|  1|  1| 0.0|0.3|
|76.71| 19|  1|  0|  7|78.42|  3|  1|  1| 0.0|0.0|
|76.71| 19|  1|  0|  7|78.42|  3|  1|  1| 0.0|0.3|
|76.71| 19|  1|  0|  7|78.42|  3|  1|  1|10.0|0.0|
|76.71| 19|  1|  0|  7|78.42|  3|  1|  1|50.0|0.0|
|78.03| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|0.0|
|78.03| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|0.3|
|78.03| 19|  1|  0|  7|78.42|  3|  1|  1|50.0|0.0|
|79.38| 19|  0|  0|  7|78.42|  3|  1|  1| 0.0|0.3|
|79.38| 19|  0|  1|  7|78.42|  

In [26]:
train.write.csv(r'C:\Users\techi\Desktop\data.csv')